In [5]:
from mp_api.client import MPRester
import csv
from api_key import APIKEY
API_KEY = APIKEY

In [15]:
import csv

passedBandgap = 'passed_bandgap.csv'
passedThermo = 'materials.csv'
data = []
MPID = []
materials = []
Bandgap = []



with open(passedBandgap, mode='r', newline='') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)
    col1_index = header.index('material_id')
    col2_index = header.index('predict_Band_gap_HSE')
    
    for row in csv_reader:
        data.append([row[col1_index], row[col2_index]])
        MPID.append(row[col1_index])
        Bandgap.append(row[col2_index])

with open(passedThermo, mode='r', newline='') as file:
    csv_reader = csv.reader(file)
    header = next(csv_reader)
    col1_index = header.index('materials')
    
    for row in csv_reader:
        materials.append(row[col1_index])
        
matching_items = []


for item in data:
    if item[0] in materials:
        matching_items.append(item)


[['mp-1008492', '2.979507183'], ['mp-1025377', '2.870952647'], ['mp-11114', '1.791958069'], ['mp-1196390', '2.477724233'], ['mp-1203699', '2.833026'], ['mp-1211440', '2.553409197'], ['mp-1216363', '1.782059794'], ['mp-1228537', '1.886107167'], ['mp-1229260', '2.039557378'], ['mp-14206', '1.961423175'], ['mp-2079', '1.721138325'], ['mp-28192', '2.894773669'], ['mp-28501', '2.253072219'], ['mp-28518', '2.595013031'], ['mp-28626', '2.611267086'], ['mp-28698', '1.960829725'], ['mp-541014', '2.235704383'], ['mp-541735', '2.075190514'], ['mp-546350', '2.602761486'], ['mp-562809', '2.221428714'], ['mp-574620', '2.975704853'], ['mp-602', '1.808474119'], ['mp-6088', '2.460748364'], ['mp-653579', '2.483570861'], ['mp-698609', '1.936351592'], ['mp-755488', '1.841813208'], ['mp-9069', '1.936658378'], ['mp-1025567', '2.141346797'], ['mp-1029730', '2.118767333'], ['mp-1192367', '2.245336761'], ['mp-1223442', '2.197690431'], ['mp-16435', '1.990902819'], ['mp-2172', '2.468452208'], ['mp-27979', '2.906

In [21]:
def process_matching_ids(matching_items):
    mat_ids = [item[0] for item in matching_items]
    return mat_ids

mat_ids = process_matching_ids(matching_items)

In [ ]:
def get_material_entries(material_ids, batch_size=6000):
    mp_entries = []
    with MPRester(API_KEY) as mpr:
        for i in range(0, len(material_ids), batch_size):
            batch_ids = material_ids[i:i + batch_size]
            material_entries = mpr.materials.search(material_ids=batch_ids)
            mp_entries.extend(material_entries)
    return mp_entries

mp_entries = get_material_entries(mat_ids)

In [23]:
print(mat_ids)

['mp-1008492', 'mp-1025377', 'mp-11114', 'mp-1196390', 'mp-1203699', 'mp-1211440', 'mp-1216363', 'mp-1228537', 'mp-1229260', 'mp-14206', 'mp-2079', 'mp-28192', 'mp-28501', 'mp-28518', 'mp-28626', 'mp-28698', 'mp-541014', 'mp-541735', 'mp-546350', 'mp-562809', 'mp-574620', 'mp-602', 'mp-6088', 'mp-653579', 'mp-698609', 'mp-755488', 'mp-9069', 'mp-1025567', 'mp-1029730', 'mp-1192367', 'mp-1223442', 'mp-16435', 'mp-2172', 'mp-27979', 'mp-554334', 'mp-558188', 'mp-567428', 'mp-567556', 'mp-570593', 'mp-673798', 'mp-684638', 'mp-10378', 'mp-1190785', 'mp-1197810', 'mp-1210261', 'mp-1211748', 'mp-29468', 'mp-553960', 'mp-560515', 'mp-567391', 'mp-5929', 'mp-672', 'mp-672352', 'mp-705895', 'mp-753395', 'mp-759923', 'mp-766127', 'mp-773453', 'mp-775866', 'mp-776315', 'mp-777048', 'mp-1029368', 'mp-1105368', 'mp-1203016', 'mp-12046', 'mp-1205840', 'mp-1212667', 'mp-1213518', 'mp-1213805', 'mp-1225986', 'mp-13287', 'mp-14406', 'mp-23412', 'mp-23514', 'mp-3762', 'mp-3772', 'mp-510226', 'mp-542662

In [20]:
composition_list = [entry.composition_reduced for entry in mp_entries if hasattr(entry, 'composition_reduced')] 

In [25]:
def process_matching_ids(matching_items):
    mat_bandgap = [item[1] for item in matching_items]
    return mat_bandgap

mat_bandgap = process_matching_ids(matching_items)

In [26]:
electroneg = []

for i in composition_list:
    electroneg.append(i.average_electroneg)

In [33]:
mat_bandgaps = [float(bg) for bg in mat_bandgap]

In [50]:
cbm = []
vbm = []

for i,x in zip(mat_bandgaps,electroneg):

    bandcorrection = i/2
    cbm_pre = x + bandcorrection
    vbm_pre = x - bandcorrection
    cbm_minusvac = cbm_pre - 4.5
    vbm_minusvac = vbm_pre - 4.5

    cbm.append(cbm_minusvac)
    vbm.append(vbm_minusvac)

In [51]:
combined_lists = list(zip(mat_ids, mat_bandgaps, cbm, vbm))

In [58]:
CO_reduction_met = [item for item in combined_lists if item[2] >= 0.12]
CHOOH_reduction_met = [item for item in combined_lists if 0 <= item[2] < 0.12]
CH4_reduction_met = [item for item in combined_lists if -0.17 <= item[2] < 0]

In [59]:
print(len(CO_reduction_met))
print(len(CHOOH_reduction_met))
print(len(CH4_reduction_met))

79
56
150


In [64]:
CO_list = [item + ("CO",) for item in CO_reduction_met]
CHOOH_list = [item + ("CHOOH",) for item in CHOOH_reduction_met]
CH4_list = [item + ("CH4",) for item in CH4_reduction_met]

In [65]:
BandAlign_pass_list = []
BandAlign_pass_list.extend(CO_list)
BandAlign_pass_list.extend(CHOOH_list)
BandAlign_pass_list.extend(CH4_list)


In [66]:
print(len(BandAlign_pass_list))

285


In [67]:
import csv

filename = 'BandAlign_pass_list.csv'
headers = ['MPID', 'BandGap', 'CBM', 'VBM', 'Product']

with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(headers)
    writer.writerows(BandAlign_pass_list)
